<a href="https://colab.research.google.com/github/HirenOrdex/ets_doc/blob/main/_swagger_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/HirenOrdex/ets_doc.git

Cloning into 'ets_doc'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 5), reused 7 (delta 2), pack-reused 0
Receiving objects: 100% (18/18), 517.74 KiB | 3.38 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [13]:
import os
import json

def process_properties(properties, required_properties, indent_level=0):
    """
    Recursively processes the properties of a schema object to generate the correct Swagger comments.
    """
    indent = " " * indent_level
    comment = ""

    for prop_name, prop_value in properties.items():
        prop_type = prop_value.get("type", "object")
        required = prop_name in required_properties
        comment += f"{indent}{prop_name}:\n"
        comment += f"{indent}  type: {prop_type}\n"
        comment += f"{indent}  required: {required}\n"
        if prop_type == "object":
            if "properties" in prop_value:
                comment += f"{indent}  properties:\n"
                comment += process_properties(prop_value["properties"], prop_value.get('required', []), indent_level + 4)
        elif prop_type == "array":
            if "items" in prop_value and "properties" in prop_value["items"]:
                comment += f"{indent}  items:\n"
                comment += process_properties(prop_value["items"]["properties"], prop_value["items"].get('required', []), indent_level + 4)

    return comment

def generate_swagger_comment(path_key, method_key, method_value):
    """
    Generate Swagger comment block for a single endpoint.
    """
    tags = method_value.get('tags', [])
    summary = method_value.get('summary', '')
    parameters = method_value.get('parameters', [])
    request_body = method_value.get('requestBody', {})
    responses = method_value.get('responses', {})

    comment = "/**\n"
    comment += f" * @swagger\n"
    comment += f" * paths:\n"
    comment += f" *   {path_key}:\n"
    comment += f" *     {method_key}:\n"
    comment += f" *       tags:\n"
    for tag in tags:
        comment += f" *         - {tag}\n"
    comment += f" *       summary: {summary}\n"

    # Parameters
    if parameters:
        comment += f" *       parameters:\n"
        for param in parameters:
            comment += f" *         - name: {param['name']}\n"
            comment += f" *           in: {param['in']}\n"
            comment += f" *           required: {param.get('required', False)}\n"
            comment += f" *           schema:\n"
            if 'schema' in param:
                comment += f" *             type: {param['schema'].get('type', 'object')}\n"
                if 'properties' in param['schema']:
                    comment += f" *             properties:\n"
                    comment += process_properties(param['schema']['properties'], param['schema'].get('required', []), 16)

    # Request Body
    if request_body:
        content = request_body.get('content', {})
        for content_type, content_details in content.items():
            schema = content_details.get('schema', {})
            if schema:
                comment += f" *       requestBody:\n"
                comment += f" *         required: true\n"
                comment += f" *         content:\n"
                comment += f" *           {content_type}:\n"
                comment += f" *             schema:\n"
                comment += f" *               type: {schema.get('type', 'object')}\n"
                if 'properties' in schema:
                    comment += f" *               properties:\n"
                    comment += process_properties(schema["properties"], schema.get('required', []), 18)

    # Responses
    if responses:
        comment += f" *       responses:\n"
        for code, resp in responses.items():
            comment += f" *         '{code}':\n"
            comment += f" *           description: {resp.get('description', '')}\n"

    comment += f" */\n"
    return comment

def process_folder(folder_path):
    # Ensure folder path ends with a slash
    if not folder_path.endswith('/'):
        folder_path += '/'

    # List all files in the folder
    try:
        files = os.listdir(folder_path)
        print(f"Files in the folder '{folder_path}': {files}")
    except Exception as e:
        print(f"Error accessing folder '{folder_path}': {e}")
        return

    # Filter JSON files
    json_files = [f for f in files if f.endswith('.json')]
    if not json_files:
        print(f"No JSON files found in the folder '{folder_path}'.")
        return

    # Process each JSON file
    for json_file in json_files:
        file_path = os.path.join(folder_path, json_file)
        print(f"Generating Swagger comments for {file_path}...")

        # Generate comments for the JSON file
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                swagger_data = json.load(f)

                comments = []
                paths = swagger_data.get('paths', {})
                for path_key, path_value in paths.items():
                    for method_key, method_value in path_value.items():
                        comment = generate_swagger_comment(path_key, method_key, method_value)
                        comments.append(comment)

                # Write comments to a txt file
                txt_file_path = os.path.splitext(file_path)[0] + '_comments.txt'
                with open(txt_file_path, 'w', encoding='utf-8') as txt_file:
                    for comment in comments:
                        txt_file.write(comment)

                print(f"Comments saved to {txt_file_path}")
        except Exception as e:
            print(f"Error processing file '{file_path}': {e}")

if __name__ == "__main__":
    api_folder = "/content/ets_doc/api"  # Replace with your API folder path

    if os.path.exists(api_folder):
        process_folder(api_folder)
    else:
        print(f"API folder '{api_folder}' not found.")


Files in the folder '/content/ets_doc/api/': ['driver.json', '.ipynb_checkpoints', 'driver_comments.txt', 'trip_comments.txt', 'auth_comments.txt', 'tyre-.json', 'trip.json', 'auth.json']
Generating Swagger comments for /content/ets_doc/api/driver.json...
Comments saved to /content/ets_doc/api/driver_comments.txt
Generating Swagger comments for /content/ets_doc/api/tyre-.json...
Comments saved to /content/ets_doc/api/tyre-_comments.txt
Generating Swagger comments for /content/ets_doc/api/trip.json...
Comments saved to /content/ets_doc/api/trip_comments.txt
Generating Swagger comments for /content/ets_doc/api/auth.json...
Comments saved to /content/ets_doc/api/auth_comments.txt
